In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [4]:
df=pd.read_excel('协方差训练数据.xlsx')

df['TradingDate'] = pd.to_datetime(df['TradingDate'])

# Step 3: 将日期转换为 1~24 的月序号
# 基准：2021年1月是第1个月
df['Month_Index'] = (df['TradingDate'].dt.year - 2021) * 12 + df['TradingDate'].dt.month

print("成功将 TradingDate 转换为 Month_Index (1~24)")

# Step 4: 其他列处理
df['Symbol'] = df['Symbol'].astype(str)
df['ClosePrice'] = pd.to_numeric(df['ClosePrice'], errors='coerce')

# Step 5: 去重（确保每只股票每个月只有一个价格）
df = df.drop_duplicates(subset=['Symbol', 'Month_Index'], keep='last')

# Step 6: 按股票分组计算月度收益率
all_returns = []

for symbol in df['Symbol'].unique():
    stock_df = df[df['Symbol'] == symbol].copy()
    stock_df = stock_df.sort_values('Month_Index')
    
    # 补全 1~24 个月（可选，避免跳月导致收益率错位）
    full_months = pd.DataFrame({'Month_Index': range(1, 25)})
    stock_df = full_months.merge(stock_df, on='Month_Index', how='left')
    
    # 计算收益率
    returns = stock_df['ClosePrice'].pct_change()
    
    result = pd.DataFrame({
        'Symbol': symbol,
        'Month_Index': stock_df['Month_Index'],
        'Monthly_Return': returns.values
    })
    all_returns.append(result)

# 合并结果
final_df = pd.concat(all_returns, ignore_index=True)

# 保存
final_df.to_excel('修正训练数据.xlsx', index=False)

print("✅ 完成！已保存到 修正训练数据.xlsx")

成功将 TradingDate 转换为 Month_Index (1~24)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_5440\3687415105.py:30: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = stock_df['ClosePrice'].pct_change()
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5440\3687415105.py:30: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = stock_df['ClosePrice'].pct_change()
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5440\3687415105.py:30: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or 

✅ 完成！已保存到 修正训练数据.xlsx


In [6]:

# Step 1: 读取原始数据
df = pd.read_excel('修正训练数据.xlsx')  # 替换为你的文件名

# Step 2: 确保数据类型正确
df['Symbol'] = df['Symbol'].astype(str)
df['Month_Index'] = df['Month_Index'].astype(int)

# Step 3: 按 Symbol 和 Month_Index 重置索引，确保唯一性
df = df.drop_duplicates(subset=['Symbol', 'Month_Index'], keep='last')

# Step 4: 转为宽格式（pivot）
wide_df = df.pivot(index='Month_Index', columns='Symbol', values='Monthly_Return')

# Step 5: 重置索引，使 Month_Index 成为普通列
wide_df = wide_df.reset_index()

# Step 6: 保存到新 Excel 文件
wide_df.to_excel('宽泛修正训练数据.xlsx', index=False)

print("✅ 宽格式数据已保存到 宽泛修正训练数据.xlsx")

✅ 宽格式数据已保存到 宽泛修正训练数据.xlsx


In [8]:
# 1. 读取 Excel 文件
file_path = "宽泛修正训练数据.xlsx"
df = pd.read_excel(file_path, header=None)  # 无标题行

# 2. 提取收益率数据：去掉第一列（时间点），只保留股票收益率
returns_df = df.iloc[1:, 1:]  # 所有行，从第2列开始

# 3. 检查形状
print("数据形状（时间点 × 股票数）:", returns_df.shape)

# 4. 计算样本协方差矩阵（按列：每列是一只股票）
cov_matrix = returns_df.cov()  # 默认 ddof=1，即样本协方差

print("协方差矩阵形状:", cov_matrix.shape)
print("协方差矩阵前5行5列:\n", cov_matrix.iloc[:5, :5])

# 5. （可选）保存协方差矩阵回 Excel
cov_matrix.to_excel("协方差矩阵_499x499.xlsx")
print("✅ 协方差矩阵已保存为 '协方差矩阵_499x499.xlsx'")

数据形状（时间点 × 股票数）: (23, 499)
协方差矩阵形状: (499, 499)
协方差矩阵前5行5列:
           1         2         3         4         5
1  0.009077  0.000519  0.001448  0.001692  0.002139
2  0.000519  0.005456  0.002090  0.000912  0.000424
3  0.001448  0.002090  0.004316  0.002827  0.002268
4  0.001692  0.000912  0.002827  0.006290  0.002882
5  0.002139  0.000424  0.002268  0.002882  0.003932
✅ 协方差矩阵已保存为 '协方差矩阵_499x499.xlsx'
